In [ ]:
import os
import pandas as pd
from datetime import datetime 

In [ ]:
# Set the working directory
current_dir = os.getcwd()
print('current_dir: ',current_dir)
parent_dir = os.path.dirname(current_dir)

os.chdir(parent_dir)

print('Current working directory:', os.getcwd())

In [ ]:
fileName = "se4g_dashboard_dataset.csv"
folder_out = 'data_prova'
full_path = os.path.join(folder_out, fileName)

country = {'AD': 'Andorra', 
        'SE':'Sweden', 
        'DE':'Germany', 
        'CY':'Undefined', 
        'BE':'Belgium', 
        'FI':'Finland', 
        'ES':'Spain', 
        'CZ':'Czech Republic', 
        'BG':'Bulgaria', 
        'BA':'Bosnia and Herzegovina', 
        'EE':'Estonia', 
        'CH':'Switzerland',
        'AT':'Austria', 
        'DK':'Denmark'}

if os.path.isfile(full_path):
    dashboard_df = pd.read_csv(full_path)
    # Convert 'value_datetime_end' to datetime objects and extract the day
    datetime_objects = dashboard_df['value_datetime_end'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S%z'))

    dashboard_df['month_day'] = datetime_objects.apply(lambda x: x.strftime('%m%d')).astype('int64')
    
    # Compute daily mean of 'value_numeric' for each 'pollutant' and 'network_countrycode'
    daily_mean = dashboard_df.groupby(['pollutant', 'network_countrycode', 'day'])['value_numeric'].mean().reset_index()
    
    # Merge the daily mean back to the original dataframe
    dashboard_df = dashboard_df.merge(daily_mean, on=['pollutant', 'network_countrycode', 'day'], suffixes=('', '_mean'))    

    dashboard_df['country'] = dashboard_df['network_countrycode'].map(country)
    dashboard_df = dashboard_df[['pollutant', 'country', 'month_day', 'value_numeric_mean','value_datetime_begin']].copy()

    dashboard_df = dashboard_df.drop_duplicates().reset_index(drop=True)
    dashboard_df = dashboard_df.sort_values('month_day')
    dashboard_df.to_csv('data_prova/se4g_dashboard_dataset_prova.csv', index=False)
    
else: 
        print("Dataset ",full_path," does not exist")
        print(os.getcwd())